# Bayes ablations: Distance × English

This notebook runs a 2×2 grid of Bayesian model variants:

- **Distance** covariate: on/off
- **English proficiency** covariate: on/off (`english_speakers_pct` from `raw/wikipedia_eng_lng_pop.csv`)

Optionally it repeats each variant across multiple random seeds.

Outputs:
- Writes run CSVs (+ sidecar `.meta.json`) into **repo-root** `data/runs/`
- Use `bayes_streamlit.py` → **Compare** tab to inspect deltas.



In [2]:
import os
import sys

import asyncio
import pandas as pd

# Ensure repo root is on sys.path so `import src...` works even when Jupyter's cwd is `notebooks/`.
sys.path.insert(0, os.path.abspath(".."))

from src.pipeline import (
    DatasetBuildConfig,
    build_dataset,
    run_bayes,
    run_heuristic,
    write_run_artifacts,
)

# -----------------
# Settings
# -----------------
LABEL_BASE = "nb-ablate-dist-eng"
# Always write artifacts into the repo-root data/runs, even if the notebook cwd is `notebooks/`.
RUNS_DIR = os.path.abspath(os.path.join("..", "data", "runs"))

# For a "proper" run, bump these up (this notebook will take longer).
DRAWS = 2000
TUNE = 2000
TARGET_ACCEPT = 0.95
HDI_PROB = 0.9

# Multiple seeds help check stability. Keep small at first (e.g. 2–3 seeds).
SEEDS = [0, 1, 2]

# Variants: 2×2 ablation grid
VARIANTS = [
    {"use_distance": True, "use_english": True, "tag": "dist1_eng1"},
    {"use_distance": True, "use_english": False, "tag": "dist1_eng0"},
    {"use_distance": False, "use_english": True, "tag": "dist0_eng1"},
    {"use_distance": False, "use_english": False, "tag": "dist0_eng0"},
]

# -----------------
# Build dataset once
# -----------------
# Build with BOTH features present so the model can ablate covariates cleanly.
# (When a covariate is disabled, the Bayes model simply doesn't use that column.)
df = await build_dataset(
        DatasetBuildConfig(dataset_csv=None, use_distance=True, use_english=True)
    )


# Heuristic output is useful to keep in the CSV for comparing against Bayes.
heur = run_heuristic(
    df,
    use_language_factor=False,
    language_english_factor=1.25,
    language_euro_latin_factor=1.0,
    language_other_factor=0.75,
)

# -----------------
# Run grid
# -----------------
rows = []

for v in VARIANTS:
    for seed in SEEDS:
        label = f"{LABEL_BASE}-{v['tag']}-seed{seed}"

        bayes = run_bayes(
            df,
            use_distance=bool(v["use_distance"]),
            use_english=bool(v["use_english"]),
            draws=int(DRAWS),
            tune=int(TUNE),
            target_accept=float(TARGET_ACCEPT),
            seed=int(seed),
            hdi_prob=float(HDI_PROB),
        )

        out = pd.merge(heur, bayes, on="alpha_3", how="left")
        csv_path, meta_path = write_run_artifacts(
            out,
            runs_dir=RUNS_DIR,
            label=label,
            meta={
                "kind": "bayes_run",
                "source": "notebook",
                "variant": v,
                "seed": int(seed),
                "draws": int(DRAWS),
                "tune": int(TUNE),
                "target_accept": float(TARGET_ACCEPT),
                "hdi_prob": float(HDI_PROB),
            },
        )

        # Pull a few run-level summaries from the CSV (coefficients are duplicated per-row).
        df_run = pd.read_csv(csv_path)
        summary_row = df_run.iloc[0]

        rows.append(
            {
                "label": label,
                "csv": csv_path,
                "meta": meta_path,
                "use_distance": bool(v["use_distance"]),
                "use_english": bool(v["use_english"]),
                "seed": int(seed),
                "beta_distance": summary_row.get("bayes_beta_log1p_uk_distance_km_mean"),
                "beta_english": summary_row.get("bayes_beta_english_speakers_rate_mean"),
                "rhat_max": summary_row.get("bayes_rhat_max"),
                "ess_bulk_min": summary_row.get("bayes_ess_bulk_min"),
                "train_n": summary_row.get("bayes_train_n"),
            }
        )

print(f"Wrote {len(rows)} runs into: {RUNS_DIR}")

summary = pd.DataFrame(rows)
summary.sort_values(["use_distance", "use_english", "seed"]).reset_index(drop=True)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]
Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 3 seconds.
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]
Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 3 seconds.
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]
Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 3 seconds.
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]
Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 2 seconds.
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]
Sampling 4 chains for 2_000 tune and 2_000 draw 

Wrote 12 runs into: /Users/joshuamason/git/renc/data/runs


,label,csv,meta,use_distance,use_english,seed,beta_distance,beta_english,rhat_max,ess_bulk_min,train_n
0,nb-ablate-dist-eng-dist0_eng0-seed0,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,False,False,0,NaN,NaN,1.53,7.0,22.0
1,nb-ablate-dist-eng-dist0_eng0-seed1,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,False,False,1,NaN,NaN,1.00,3485.0,22.0
2,nb-ablate-dist-eng-dist0_eng0-seed2,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,False,False,2,NaN,NaN,1.53,7.0,22.0
3,nb-ablate-dist-eng-dist0_eng1-seed0,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,False,True,0,NaN,-4.187509,1.00,3458.0,22.0
4,nb-ablate-dist-eng-dist0_eng1-seed1,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,False,True,1,NaN,-4.188694,1.00,2950.0,22.0
5,nb-ablate-dist-eng-dist0_eng1-seed2,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,False,True,2,NaN,-4.187840,1.00,3296.0,22.0
6,nb-ablate-dist-eng-dist1_eng0-seed0,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,True,False,0,3.219274,NaN,1.00,3866.0,22.0
7,nb-ablate-dist-eng-dist1_eng0-seed1,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,True,False,1,3.217575,NaN,1.00,4138.0,22.0
8,nb-ablate-dist-eng-dist1_eng0-seed2,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,True,False,2,3.217897,NaN,1.00,3889.0,22.0
9,nb-ablate-dist-eng-dist1_eng1-seed0,/Users/joshuamason/git/renc/data/runs/nb-ablat...,/Users/joshuamason/git/renc/data/runs/nb-ablat...,True,True,0,2.619907,-2.256896,1.00,3769.0,22.0


In [ ]:
# -----------------
# Stability checks across seeds
# -----------------
# What to look for:
# - Convergence: rhat_max ~<= 1.01 and ess_bulk_min reasonably large (rule of thumb: > 400)
# - Stability: high Spearman correlation of ranks/probabilities across seeds; high top-N overlap

from itertools import combinations

TOP_N = 30

# Load all run CSVs once
runs = {}
for r in rows:
    runs[r["label"]] = pd.read_csv(r["csv"])

# Helper: get a comparable series by alpha_3
def _series(df: pd.DataFrame, col: str) -> pd.Series:
    s = df.set_index("alpha_3")[col]
    return pd.to_numeric(s, errors="coerce")

stability_rows = []

for (use_distance, use_english), grp in summary.groupby(["use_distance", "use_english"], dropna=False):
    labels = grp["label"].tolist()

    # Pairwise comparisons across seeds
    for a, b in combinations(labels, 2):
        A = runs[a]
        B = runs[b]

        # Spearman via ranking (avoid scipy dependency)
        p1_a = _series(A, "bayes_p_one_mean")
        p1_b = _series(B, "bayes_p_one_mean")
        common = p1_a.index.intersection(p1_b.index)

        ra = p1_a.loc[common].rank(method="average")
        rb = p1_b.loc[common].rank(method="average")
        spearman_p1 = ra.corr(rb)

        # Top-N overlap by bayes_rank
        top_a = (
            A.sort_values("bayes_rank", ascending=True)
            .head(TOP_N)["alpha_3"]
            .astype(str)
            .tolist()
        )
        top_b = (
            B.sort_values("bayes_rank", ascending=True)
            .head(TOP_N)["alpha_3"]
            .astype(str)
            .tolist()
        )
        overlap = len(set(top_a).intersection(set(top_b)))

        stability_rows.append(
            {
                "use_distance": bool(use_distance),
                "use_english": bool(use_english),
                "pair": f"{a} vs {b}",
                "spearman_rank(p1)": float(spearman_p1) if pd.notna(spearman_p1) else None,
                f"top{TOP_N}_overlap": int(overlap),
            }
        )

stability = pd.DataFrame(stability_rows)
stability.sort_values(["use_distance", "use_english", f"top{TOP_N}_overlap"], ascending=[True, True, False])

# Coefficient + diagnostic stability summary (per variant)
coef_cols = ["beta_distance", "beta_english", "rhat_max", "ess_bulk_min"]
(
    summary.groupby(["use_distance", "use_english"], dropna=False)[coef_cols]
    .agg(["mean", "std", "min", "max"])
    .round(3)
)

# Optional: per-country probability instability (std across seeds) for one variant
# Pick a variant by toggling these:
SHOW_VARIANT = {"use_distance": True, "use_english": True}

variant_labels = summary[
    (summary["use_distance"] == SHOW_VARIANT["use_distance"])
    & (summary["use_english"] == SHOW_VARIANT["use_english"])
]["label"].tolist()

if len(variant_labels) >= 2:
    stacked = []
    for lab in variant_labels:
        df_run = runs[lab][["alpha_3", "country_name", "bayes_p_one_mean", "bayes_rank"]].copy()
        df_run = df_run.rename(
            columns={
                "bayes_p_one_mean": f"p1__{lab}",
                "bayes_rank": f"rank__{lab}",
            }
        )
        stacked.append(df_run)

    merged = stacked[0]
    for nxt in stacked[1:]:
        merged = merged.merge(nxt, on=["alpha_3", "country_name"], how="inner")

    p1_cols = [c for c in merged.columns if c.startswith("p1__")]
    merged["p1_std_across_seeds"] = merged[p1_cols].std(axis=1)

    merged.sort_values("p1_std_across_seeds", ascending=False).head(25)
else:
    print("Need 2+ seeds for the selected variant to compute per-country instability.")

